## 3.2 恢复模型  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;与保存模型一样，需要在构造期末尾创建一个saver节点，但在执行期开始时候不是用init节点来初始化所有变量，而是调用Saver对象上的restore()方法。继续以加州房价线性模型为例： 

**代码示例**：

```
# 导入包
# .....
# 下载并整理数据
# .....
# 数据预处理
# .....
# 构造期
# .....
# 执行期
def fetch_batch(epoch, batch_index, batch_size):
    np.random.seed(epoch * n_batches + batch_index) 
    indices = np.random.randint(m, size=batch_size)
    X_batch = scaled_housing_data_plus_bias[indices] 
    y_batch = housing.target.reshape(-1, 1)[indices] 
    return X_batch, y_batch

with tf.Session() as sess:
    saver.restore(sess, "./tmp/my_model_final.ckpt")
    
    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            sess.run(training_op, feed_dict={X:X_batch, y:y_batch})
    best_theta = theta.eval()
    print("The best theta is", best_theta)
```

In [1]:
# 导入相关包
import tensorflow as tf
import numpy as np
from sklearn.datasets import fetch_california_housing
from sklearn.preprocessing import StandardScaler
import datetime

In [2]:
# 下载及整理数据
housing = fetch_california_housing()
m,n = housing.data.shape # 获取数据的行列数
housing_data_plus_bias = np.c_[np.ones((m,1)),housing.data] # 为数据添加偏差项，即添加y=ax+b中的b

In [3]:
# 数据预处理
scaler = StandardScaler().fit(housing_data_plus_bias)
scaled_housing_data_plus_bias = scaler.transform(housing_data_plus_bias)

In [4]:
X = tf.placeholder(tf.float32, shape=(None, n+1), name="X")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")
n_epochs = 1000
batch_size = 100
n_batches= int(np.ceil(m/batch_size))
global_learning_rate = 0.01
XT = tf.transpose(X)
theta = tf.Variable(tf.random_uniform([n+1,1],-1.0,1.0),name="theta")     # * 参数 seed=42
y_pred =  tf.matmul(X, theta, name="prediction")                          # 预测值
error = y_pred-y                                                          # 误差
mse = tf.reduce_mean(tf.square(error), name="mse")                        # 均方误差(成本函数)                                 
# 调用特定的优化器求解梯度并优化
optimizer = tf.train.GradientDescentOptimizer(learning_rate = global_learning_rate)
training_op = optimizer.minimize(mse)
# 添加一个saver节点用来保存模型参数
saver = tf.train.Saver()

Instructions for updating:
Colocations handled automatically by placer.


In [5]:
def fetch_batch(epoch, batch_index, batch_size):
    np.random.seed(epoch * n_batches + batch_index) 
    indices = np.random.randint(m, size=batch_size)
    X_batch = scaled_housing_data_plus_bias[indices] 
    y_batch = housing.target.reshape(-1, 1)[indices] 
    return X_batch, y_batch

with tf.Session() as sess:
    saver.restore(sess, "./tmp/my_model_final.ckpt")
    
    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            sess.run(training_op, feed_dict={X:X_batch, y:y_batch})
    best_theta = theta.eval()
    print("The best theta is", best_theta)

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from ./tmp/my_model_final.ckpt
The best theta is [[ 0.6356673 ]
 [ 0.8377844 ]
 [ 0.10645497]
 [-0.25947902]
 [ 0.29196444]
 [ 0.00181689]
 [ 0.2128084 ]
 [-0.89034677]
 [-0.85242176]]


## 3.3 模型文件解析
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Tensorflow训练后的模型可以保存checkpoint文件或pb文件。checkpoint文件是结构与权重分离的四个文件，便于训练；pb文件则是graph_def的序列化文件，类似于caffemodel，便于发布和离线预测。官方提供freeze_grpah.py脚本来将ckpt文件转为pb文件。
1. Checkpoint保存断点文件列表，可以用来迅速查找最近一次的断点文件；  
2. meta文件是MetaGraphDef序列化的二进制文件，保存了网络结构相关的数据，包括graph_def和saver_def等；  
3. index文件为数据文件提供索引，存储的核心内容是以tensor name为键以BundleEntry为值的表格entries，BundleEntry主要内容是权值的类型、形状、偏移、校验和等信息。  
4. Index文件由data block/index block/Footer等组成，构建时主要涉及BundleWriter、TableBuilder、BlockBuilder几个类，除了BundleEntry的序列化，还涉及了tensor name的编码及优化（比如丢弃重复的前缀）和data block的snappy压缩。数据（data）文件保存所有变量的值，即网络权值。

## 注意事项
1. 在恢复模型时，需新建一个代码文件执行上述代码，如果在保存模型代码中再新建会话恢复模型会出现报错。